In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt, timedelta
from time import time

As mentioned in EDA, we'll be testing change in lapse behaviour pre- and post- GFC <br>
Instead of performing x-validation for the entire training set, we will look at model performance and feature-stability over the following contiguous periods: <br>
1. (-2007, 2008): Pre-crisis period -> we're already seeing lapse rates creeping up <br>
2. (-2008, 2009): Training period is pre-crisis, Test period is post-crisis -> big bump in policies written in test period <br>
3. (-2009, 2010): Training period includes post-crisis -> another big bump in policies written in test period, bump in lapse-rate as well <br>
4. (-2010, 2011): Sharp drop in policies written in test period, further rise in lapse-rate <br>
(-,): (train_period, test_period). Test-window is always 1y ahead. Train-window end-date slides forward to include previous test-window. Train-window start date is fixed at origin. <br><br>
In reality, test-windows should be more granular: quarterly/monthly  

### Generate the training sets

In [2]:
path=os.getcwd()+"\\data\\home_insurance_training.csv"
df_training=pd.read_csv(path)

In [3]:
df_x_validation_sets=[df_training[df_training['cover_start_year'] <= x].copy() for x in [2008, 2009, 2010, 2011]]

Label training and validation sets

In [4]:
def get_train_set_labels(df):
    year_values=df['cover_start_year'].copy()
    last_year=year_values.iloc[-1]
    df['is_test']=[1 if x == last_year else 0 for x in year_values]
    return df

In [5]:
df_labelled_sets=[get_train_set_labels(df) for df in df_x_validation_sets]

Get the number of records in each fold, the train-test splits and the train-set lapse ratios

In [6]:
def get_lapse_ratio(values):
    list_vals=list(values)
    lapse_ratio=round(sum(list_vals)/len(list_vals),2)
    return lapse_ratio

In [7]:
train_records=[len(x[x['is_test'] == 0]) for x in df_labelled_sets]
test_records=[len(x[x['is_test'] == 1]) for x in df_labelled_sets]
lapse_ratios=[get_lapse_ratio(x[x['is_test'] == 0]['is_lapsed']) for x in df_labelled_sets]
test_set_label=[x[x['is_test'] == 1]['cover_start_year'].iloc[0] for x in df_labelled_sets]

In [8]:
df_num_records=pd.DataFrame({'test_set_label': test_set_label, 'train_records': train_records, 'test_records': test_records, 'train_set_lapse_ratios': lapse_ratios})
df_num_records['train_test_split']=[round(x/(x+y),2) for x,y in zip(df_num_records['train_records'], df_num_records['test_records']) ]
df_num_records

,test_set_label,train_records,test_records,train_set_lapse_ratios,train_test_split
0,2008,83375,11502,0.21,0.88
1,2009,94877,21850,0.22,0.81
2,2010,116727,40877,0.23,0.74
3,2011,157604,31415,0.26,0.83


Train-set lapse ratios are small enough for the positive set (lapsed policies) to be considered 'unbalanced'. We will check both cases. 

#### Write out the training sets

In [9]:
base_path=os.getcwd()+"\\data\\folds\\"
[df.set_index('is_lapsed').to_csv(base_path+str(label)+".csv") for df,label in zip(df_labelled_sets, df_num_records['test_set_label'])]

[None, None, None, None]